In [54]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

# Step 1: Make a request to the webpage URL and extract data from the fourth table
url = 'https://www.chancedegol.com.br/br23.htm'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table')[7]

# Step 2: Loop through the rows of the table and extract the data from the cells
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append(cols)

    
# Step 3:Define file path

    # Define the absolute path to the data directory (where you want to save the CSV file)
data_dir = 'C:\\Users\\Cristina Jorge\\OneDrive\\Documentos\\GitHub\\Back-End-Elo\\data'


    # Specify the absolute path to the CSV file within the data directory
csv_file_path = os.path.join(data_dir, 'dataBRA23raw.csv')

    # Ensure the data directory exists; create it if not
os.makedirs(data_dir, exist_ok=True)

# Step 4: Write the data to a CSV file using the absolute path
with open(csv_file_path, 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerows(data)




In [55]:
df = pd.read_csv(csv_file_path)
df

,Data,Mandante,Unnamed: 2,Visitante,Vitória doMandante,Empate,Vitória doVisitante
0,15/04/2023,Botafogo,2x1,São Paulo,36.9 %,27.9 %,35.2 %
1,15/04/2023,Palmeiras,2x1,Cuiabá,74.0 %,18.6 %,7.4 %
2,15/04/2023,Red Bull Bragantino,2x1,Bahia,56.7 %,23.7 %,19.6 %
3,15/04/2023,Atlético MG,1x2,Vasco,55.1 %,25.9 %,19.0 %
4,15/04/2023,Athletico PR,2x0,Goiás,61.1 %,22.7 %,16.2 %
...,...,...,...,...,...,...,...
375,06/12/2023,Cruzeiro,1x1,Palmeiras,24.5 %,33.2 %,42.3 %
376,06/12/2023,Internacional,3x1,Botafogo,39.7 %,26.7 %,33.6 %
377,06/12/2023,Coritiba,0x2,Corinthians,33.7 %,24.0 %,42.3 %
378,06/12/2023,Bahia,4x1,Atlético MG,27.4 %,27.3 %,45.3 %


In [56]:
filt = ((df['Mandante'] == 'Botafogo') & (df['Visitante'] == 'São Paulo'))
df[filt]

,Data,Mandante,Unnamed: 2,Visitante,Vitória doMandante,Empate,Vitória doVisitante
0,15/04/2023,Botafogo,2x1,São Paulo,36.9 %,27.9 %,35.2 %


In [57]:
df = pd.read_csv(csv_file_path)
#Step 5: cleaning data frame

df.rename(columns={'Mandante':'Casa','Visitante':'Fora'}, inplace =True)
    #changes dtype to datetime in first dataframe
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
    #creates a new column with the for the competition name

df = df.assign(Comp='Série A')

    #separets the results of second dataframe into new columns
new = df["Unnamed: 2"].str.split("x", n = 1, expand = True)
df["GC"]= new[0] 
df["GF"]= new[1]
    #creates new column with competition name

    #comparing scores to display the results in the second dataframe
df.loc[df['GC'] == df['GF'], 'Res'] = 'E'
df.loc[df['GC'] > df['GF'], 'Res'] = 'VC' 
df.loc[df['GC'] < df['GF'], 'Res'] = 'VV'
    # create a new column to store the season information in the second data frame
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
df['Temp'] = df['Data'].dt.year
    #organizes de columns of the sacond dataframe
new_order = ['Comp','Temp','Data','Casa','GC', 'GF','Fora','Res']
df= df.reindex(columns=new_order)



df.replace({'América MG': 'América-MG',
            'Atlético MG': 'Atlético-MG',
            'Athletico PR': 'Athletico-PR',                   
            }, inplace=True)

    #drops unvalid data
df.dropna(axis=0, how='all', inplace=True)
    #sort values by date
df = df.sort_values(by='Data')

    #atributes a pontuation to each game
df['PC'] = df.apply(lambda x: 3 if x['Res'] == 'VC' else
                              1 if x['Res'] == 'E' else 0, axis=1)
df['PF'] = df.apply(lambda x: 3 if x['Res'] == 'VV' else
                              1 if x['Res'] == 'E' else 0, axis=1)

    #atributes a number of games for each season
df['J'] = df.groupby('Temp').cumcount() + 1
    #corrects any possible issue before saving as new file
df = df.dropna()
df

C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_2772\3355884856.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_2772\3355884856.py:6: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_2772\3355884856.py:22: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.or

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
0,Série A,2023,2023-04-15,Botafogo,2,1,São Paulo,VC,3,0,1
1,Série A,2023,2023-04-15,Palmeiras,2,1,Cuiabá,VC,3,0,2
2,Série A,2023,2023-04-15,Red Bull Bragantino,2,1,Bahia,VC,3,0,3
3,Série A,2023,2023-04-15,Atlético-MG,1,2,Vasco,VV,0,3,4
4,Série A,2023,2023-04-15,Athletico-PR,2,0,Goiás,VC,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...
372,Série A,2023,2023-12-06,São Paulo,1,0,Flamengo,VC,3,0,376
371,Série A,2023,2023-12-06,Vasco,2,1,Red Bull Bragantino,VC,3,0,377
370,Série A,2023,2023-12-06,Fluminense,2,3,Grêmio,VV,0,3,378
373,Série A,2023,2023-12-06,Santos,1,2,Fortaleza,VV,0,3,379


In [58]:
#Step 6:
    # Specify the absolute path to the new CSV file within the data directory
cleaned_csv_file_path = os.path.join(data_dir, 'serieA_2023_matches.csv')
df.to_csv(cleaned_csv_file_path, index=False, encoding='utf-8-sig')

    # Save the cleaned DataFrame to a new CSV file
df.to_csv(cleaned_csv_file_path, index=False)

    # Display the cleaned DataFrame
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
0,Série A,2023,2023-04-15,Botafogo,2,1,São Paulo,VC,3,0,1
1,Série A,2023,2023-04-15,Palmeiras,2,1,Cuiabá,VC,3,0,2
2,Série A,2023,2023-04-15,Red Bull Bragantino,2,1,Bahia,VC,3,0,3
3,Série A,2023,2023-04-15,Atlético-MG,1,2,Vasco,VV,0,3,4
4,Série A,2023,2023-04-15,Athletico-PR,2,0,Goiás,VC,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...
372,Série A,2023,2023-12-06,São Paulo,1,0,Flamengo,VC,3,0,376
371,Série A,2023,2023-12-06,Vasco,2,1,Red Bull Bragantino,VC,3,0,377
370,Série A,2023,2023-12-06,Fluminense,2,3,Grêmio,VV,0,3,378
373,Série A,2023,2023-12-06,Santos,1,2,Fortaleza,VV,0,3,379
